# Datasets

In [ ]:
#| default_exp datasets

In [ ]:
#| export
from torch.utils.data import DataLoader, default_collate
from operator import itemgetter
from fastcore import docments

In [ ]:
#| hide
from datasets import load_dataset, load_dataset_builder
import torchvision.transforms as TF

ds = load_dataset('fashion_mnist')

Found cached dataset fashion_mnist (/Users/olly/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1)


  0%|          | 0/2 [00:00<?, ?it/s]

## Convenience functions

In [ ]:
#| export
def inplace(f):
    """
        Performs dataset operations in place. To be used as a
        decorator
    """
    def _f(b):
        f(b)
        return b
    return _f

In [ ]:
@inplace
def transform(b): b['image'] = [TF.to_tensor(i) for i in b['image']]

In [ ]:
#| hide
tds = ds.with_transform(transform)

In [ ]:
#| export
def collate_dict(ds):
    """
        Takes a dataset dictionary and returns datasets as a tuple. Usually
        used when calling the Pytorch DataLoader object.
    """
    get = itemgetter(*ds.features)
    def _f(b): return get(default_collate(b))
    return _f

## Dataloaders

In [ ]:
#| export
class DataLoaders:
    """
        Establishes DataLoader objects for training and validation sets,
        and optionally returns them as a tuple.
    """
    def __init__(
        self, 
        *dsd # 
    ): 
        self.train, self.valid = dsd[:2]
    
    @classmethod
    def from_dd(
        cls, 
        dd, # Dataset dict object (works with hugging face datasets) 
        batch_size: int, # Batch size for the dataloader
        as_tuple: bool=True # If true, returns a tuple of dataloaders like (train, valid)
    ):
        return cls(*[DataLoader(ds, batch_size, num_workers=4, collate_fn=collate_dict(ds) if as_tuple else default_collate) for ds in dd.values()])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()